In [1]:
imgs_folder = '/home/jasp/autocls/img'

shp = '/home/jasp/autocls/coimbra_original.shp'
lulc_col = 'code'

refrst = '/home/jasp/autocls/rst_coimbra.tif'

out = '/home/jasp/autocls/'

In [2]:
import os
from glass.cons.stl import bandsmap
from glass.pys.oss import mkdir, lst_ff, lst_fld
from glass.pys.tm import now_as_str
from glass.wenv.grs import run_grass

In [3]:
stats = {
    'number' : 'n', 'minimum' : 'min', 'maximum': 'max',
    'range' : 'ran', 'average' : 'avg', 'stddev' : 'std'
}

_bmap = bandsmap()

bmap = {b.lower() : _bmap[b] for b in _bmap}

In [4]:
imgs = lst_fld(imgs_folder)

In [5]:
img_bands = {}
for img in imgs:
    img_bands[img] = lst_ff(img, file_format='.tif')

In [6]:
ws, loc = mkdir(os.path.join(
    out, now_as_str(utc=True)
), overwrite=True), 'loc_istats'

gb = run_grass(ws, location=loc, srs=refrst)

import grass.script.setup as gsetup

gsetup.init(gb, ws, loc, 'PERMANENT')

'/tmp/tmps7dfb3_h'

In [7]:
from glass.it.shp  import shp_to_grs, grs_to_shp
from glass.it.rst  import rst_to_grs
from glass.rst.zon import rst_stats_by_feat
from glass.gp.gen import dissolve
from glass.tbl.grs import add_table
from glass.tbl.col import rn_cols

In [8]:
for k in img_bands:
    grsshp = shp_to_grs(shp, olyr=os.path.basename(k))

    dissshp = dissolve(grsshp, f'diss_{grsshp}', lulc_col, api='grass')

    #add_table(dissshp, None, lyrN=1, asCMD=True, keyp=lulc_col)
    
    for r in img_bands[k]:
        gband = rst_to_grs(r)
    
        for b in bmap:
            if b in gband:
                prefix = bmap[b]
                break
    
        colsrn = {f"{prefix}_{k}" : f"{prefix}_{stats[k]}" for k in stats}
    
        rst_stats_by_feat(dissshp, gband, prefix, list(stats.keys()))
        rn_cols(dissshp, colsrn, api='grass')
    
        rst_stats_by_feat(grsshp, gband, prefix, list(stats.keys()))
        rn_cols(grsshp, colsrn, api="grass")
    
    grs_to_shp(grsshp, os.path.join(out, f"{grsshp}.shp"), 'area')
    grs_to_shp(dissshp, os.path.join(out, f'{grsshp}_class.shp'), 'area')